In [1]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)
dls = ImageDataLoaders.from_name_re(path, get_image_files(path/'images'), pat='(.+)_\d+.jpg', item_tfms=Resize(460), batch_tfms=aug_transforms(size=224, min_scale=0.75), bs=128)
learn = cnn_learner(dls, models.resnet50, metrics=accuracy)
learn.fine_tune(5)
learn.path = Path('.')
learn.export()

/home/camaro/anaconda3/envs/gradio/lib/python3.8/site-packages/torch/_tensor.py:1051: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  ret = func(*args, **kwargs)


epoch,train_loss,valid_loss,accuracy,time
0,1.359868,0.263231,0.918809,00:27


epoch,train_loss,valid_loss,accuracy,time
0,0.338471,0.239413,0.929635,00:30
1,0.291378,0.262474,0.927605,00:30
2,0.219857,0.189212,0.947226,00:30
3,0.148120,0.188948,0.952639,00:30
4,0.100334,0.183129,0.951962,00:29


In [2]:
learn = load_learner('export.pkl')

In [3]:
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [4]:
import gradio as gr

title = "Pet Breed Classifier"
description = "A pet breed classifier trained on the Oxford Pets dataset"
interpretation='default'
examples = ['siamese.jpg', 'kitten.jpg']
article="<p style='text-align: center'><a href='https://dicksonneoh.com' target='_blank'>Blog post</a></p>"
enable_queue=True

gr.Interface(fn=predict,inputs=gr.inputs.Image(shape=(512, 512)),outputs=gr.outputs.Label(num_top_classes=3),title=title,description=description,article=article,examples=examples,interpretation=interpretation,enable_queue=enable_queue).launch(share=True)


Running on local URL:  http://127.0.0.1:7860/


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?